In [1]:
import random
import numpy as np
import pandas as pd
import os
import time
from datetime import datetime, timedelta

In [2]:
import mysql.connector

try:
    mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="admin",
    database="recommender"
    )
except:
    mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="admin"
    )
    mycursor.execute("CREATE DATABASE recommender")

mycursor = mydb.cursor()


In [ ]:
try:
    mycursor.execute("DROP TABLE customers")
    mycursor.execute("DROP TABLE executives")
    mycursor.execute("DROP TABLE customer_logs")
    mycursor.execute("DROP TABLE executive_logs")
except:
    pass

mycursor.execute("CREATE TABLE customer_logs (record_num INT AUTO_INCREMENT PRIMARY KEY, customer_id INT NOT NULL, executive_id INT NOT NULL, \
customer_type VARCHAR(255) NOT NULL, created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, message VARCHAR(255), account_type VARCHAR(255), \
feedback VARCHAR(255), balance INT)")
mycursor.execute("CREATE TABLE executive_logs (record_num INT AUTO_INCREMENT PRIMARY KEY, executive_id INT NOT NULL, \
executive_type VARCHAR(255) NOT NULL, created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, message VARCHAR(255), \
action VARCHAR(255))")
mycursor.execute("CREATE TABLE customers (record_num INT AUTO_INCREMENT PRIMARY KEY, customer_id INT NOT NULL, \
customer_type VARCHAR(255) NOT NULL, enrolled_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, account_type VARCHAR(255))")
mycursor.execute("CREATE TABLE executives (record_num INT AUTO_INCREMENT PRIMARY KEY, executive_id INT NOT NULL, \
executive_type VARCHAR(255) NOT NULL, enrolled_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)")

In [ ]:
client_id = 0
executive_id = 0

In [ ]:

topics = ["transferencias",
            "pagos",
            "cuenta",
            "pago automático",
            "saldo",
            "compra",
            "inversiones", 
            "fondo mutuo",
            "depósito a plazo",
            "tasa crédito", 
            "crédito",
            "deuda",
            "regularización", 
            "tarjeta",
            "cajero automático",
            "sucursal",
            "ubicación sucursal",
            "ubicación cajero automático",
            "ingresar a cuenta",
            "contraseña olvidada",
            "fraude",
            "falla app",
            "falla transferencia"]
message_templates = ["Necesito ayuda con *",
                    "Me puedes ayudar con una * en ",
                    "Quiero hacer una * en ",
                    "Necesito enviar dinero",
                    "Quiero comprar dólares",
                    "Ayúdame con *",
                    "Hace una transferencia de *",
                    "No entiendo cómo se hace *",
                    "Quiero reclamar por *",
                    "No puedo obtener ayuda con *",
                    "Olvidé cómo hacerlo con *"]

messages = []
divisas = ["Euros", "Dólares", "Yuanes", "Pesos", "Rupías", "Yenes", "Pesos argentinos", "Bolívares"]

for m in message_templates:
    if "Hace una transferencia de" not in m:
        for t in topics:
            msje = m.replace("*", t)
            if " en " in msje:
                msje += random.choice(divisas)
            if msje not in messages:
                messages.append(msje)

executive_messages = {}
actions = {}
for t in topics:
    executive_messages[t] = []
    actions[t] = []

for t in topics:
    for m in messages:
        if t in m:
            for i in range(0,10):
                if random.randint(1,5) == 1:
                    msje = "Te ayudaré con {}".format(t)
                elif random.randint(1,4) == 1:
                    msje = "Ingresé una solicitud por {}, espera {} horas".format(t, str(random.randint(24, 72)))
                elif random.randint(1,10)==1:
                    msje = "He consultado por {}".format(t)
                else:
                    msje = "Haré lo posible por lograr {}".format(t)
                if msje not in executive_messages[t]:
                    executive_messages[t].append(msje)

code = 0
encodings = {}
for t in topics:
    encodings[t] = code
    code += 1

for t in topics:
    if random.randint(1,10) == 1:
        act = "Ingreso de solicitud por {}".format(encodings[t])
    elif random.randint(1,5) == 1:
        act = "Aplica operación por {}".format(encodings[t])
    elif random.randint(1,4) == 1:
        act = "Ingresa reclamo por {}".format(encodings[t])
    else:
        act = "Lleva a cabo consulta interna {}-{} por {}".format(len(t), len(topics), encodings[t])
    if act not in actions[t]:
        actions[t].append(act)



In [ ]:
bank_type = ["Business", "Person", "Pyme", "Others"]

In [ ]:
class Cliente:
    def __init__(self, client_type, trouble_frequency, account_type) -> None:
        global client_id
        client_id += 1
        self.account = account_type
        self.id = client_id
        self.type = client_type
        self.frequency = trouble_frequency
        if self.type == "Business":
            self.purse = random.randint(10000,100000)
            self.business_importance = 3
        elif self.type == "Person":
            self.purse = random.randint(10,100)
            self.business_importance = 1
        elif self.type == "Pyme":
            self.purse = random.randint(100,1000)
            self.business_importance = 2
        else:
            self.purse = random.randint(1,10)
            self.business_importance = 0
        self.current_problem = ''
        
        self.enroll()
    
    def enroll(self) -> None:
        global mycursor
        
        # Getting the current date and time
        ts = datetime.now()

        mycursor.execute("INSERT INTO customers (record_num, customer_id, customer_type, enrolled_at, account_type) VALUES ({}, {}, \"{}\", \"{}\", \"{}\")".format(self.id, self.id, self.type, ts, self.account))
        mydb.commit()
        #time.sleep(.1)

    def send_message(self, booster) -> list:
        global messages, topics
        if random.randint(self.business_importance, 10) - self.frequency - booster < 2:
            topic = ''
            while len(topic) == 0:
                msje = random.choice(messages)
                for t in topics:
                    if t in msje:
                        self.current_problem = t
                        topic = t
            if "pago" in msje or "transferencia" in msje or "crédito" in msje or "compra" in msje or "depósito" in msje or "deuda" in msje:
                self.purse *= 1/random.randint(10, 100)
            else:
                self.purse *= (1+(1/random.randint(10, 100)))
            self.purse = int(self.purse) + 1
            ls = [self.id, msje, self.purse, self.account, self.type, self.business_importance]
            return ls
        else:
            ls = [self.id, "", self.purse, self.account, self.type, self.business_importance]
            return ls
        
    def give_feedback(self, topic) -> str:
        if topic == self.current_problem:
            self.current_problem = ''
            return "OK"
        else:
            return "NO"

In [ ]:
class Ejecutivo:
    def __init__(self, executive_type, experience) -> None:
        global executive_id
        executive_id += 1
        self.id = executive_id
        self.type = executive_type
        self.exp = experience

        self.enroll()
    
    def enroll(self) -> None:
        global mycursor, mydb
        # Getting the current date and time
        ts = datetime.now()

        mycursor.execute("INSERT INTO executives (record_num, executive_id, executive_type, enrolled_at) VALUES ({}, {}, \"{}\", \"{}\")".format(self.id, self.id, self.type, ts))
        mydb.commit()
        # time.sleep(.1)

    def respond_message(self, message, client_id, account, importance) -> list:
        global topics, actions, executive_messages
        if 10 - self.exp > 5 - importance:
            divisa = ''
            for d in divisas:
                if d in message:
                    divisa = " - Operación en " + d
            topic = random.choice(topics)
            msje = random.choice(executive_messages[topic])
            action = random.choice(actions[topic]) + divisa
            msje = "Cliente {} con cuenta tipo {}{}: ".format(client_id, account, divisa) + msje
            return [topic, msje, action]
    
        else:
            topic = ''
            for t in topics:
                if t in message:
                    topic = t
            msje = random.choice(executive_messages[topic])
            action = random.choice(actions[topic])
            return [topic, msje, action]

In [ ]:
executives = []
customerss = []
for i in range(100):
    executives.append(Ejecutivo(random.choice(bank_type), random.randint(0,10)))
for i in range(10000):
    customerss.append(Cliente(random.choice(bank_type),
     random.randint(1,10), 
     random.choice(["Corriente", "Vista", "Ahorro"]))
     )

In [ ]:
recnum = 0

def interact(customer: Cliente , executive: Ejecutivo) -> bool:
    global mycursor, mydb, recnum
    if customer.type == executive.type:
        status_ok = False
        ts1 = datetime.now() + timedelta(days=random.randint(1,365), seconds=random.randint(1,5))
        # print(customer.send_message())
        boost = 0
        weekday = ts1.weekday()
        if weekday == 0:
            boost = 7
        elif weekday == 1:
            boost = 4
        elif weekday == 2:
            boost = 3
        elif weekday == 3:
            boost = 2
        elif weekday == 4:
            boost = 5
        elif weekday == 5:
            boost = 1
        elif weekday == 6:
            boost = 0
        else:
            boost = 0
        c_id, message,  purse, account, c_type, importance = customer.send_message(boost)
        if len(message) != 0:
            while not status_ok:
                # print(recnum)
                topic, response, action = executive.respond_message(message, c_id, account, importance)
                feedback = customer.give_feedback(topic)
                mycursor.execute("INSERT INTO customer_logs (record_num, customer_id, executive_id, customer_type, created_at, message, account_type, feedback, balance) VALUES ({}, {}, {}, \"{}\", \"{}\", \"{}\", \"{}\", \"{}\", {})".format(recnum, customer.id, executive.id, customer.type, ts1, message, customer.account, feedback, customer.purse))
                mydb.commit()
                #time.sleep(1)
                ts1 += timedelta(seconds=random.randint(1,5))
                mycursor.execute("INSERT INTO executive_logs (record_num, executive_id, executive_type, created_at, message, action) VALUES ({}, {}, \"{}\", \"{}\", \"{}\", \"{}\")".format(recnum, executive.id, executive.type, ts1, response, action))
                mydb.commit()
                if feedback == "OK":
                    status_ok = True
                else:
                    pass
                recnum += 2
            return True
        return False
    else:
        return False

In [ ]:
try:
    mycursor.execute("DROP TABLE customer_logs")
    mycursor.execute("DROP TABLE executive_logs")
except:
    pass

mycursor.execute("CREATE TABLE customer_logs (record_num INT AUTO_INCREMENT PRIMARY KEY, customer_id INT NOT NULL, executive_id INT NOT NULL, \
customer_type VARCHAR(255) NOT NULL, created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, message VARCHAR(255), account_type VARCHAR(255), \
feedback VARCHAR(255), balance INT)")
mycursor.execute("CREATE TABLE executive_logs (record_num INT AUTO_INCREMENT PRIMARY KEY, executive_id INT NOT NULL, \
executive_type VARCHAR(255) NOT NULL, created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, message VARCHAR(255), \
action VARCHAR(255))")

In [3]:
from tqdm import tqdm

In [ ]:
sim_cycles = 5000
for cycle in tqdm(range(sim_cycles)):
    status = False
    while not status:
        customer = random.choice(customerss)
        executive = random.choice(executives)
        status = interact(customer, executive)

# Collect data

In [34]:
from mysql.connector import Error

def query_with_fetchone():
    global mycursor, mydb
    try:
        mydb = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="admin",
        database="recommender"
        )
    except:
        mydb = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="admin"
        )
        mycursor.execute("CREATE DATABASE recommender")

    mycursor = mydb.cursor()
    try:
        cursor = mycursor
        cursor.execute("SELECT t1.action AS executed_action, \
                        t1.created_at AS executive_message_time, \
                        t1.executive_type, \
                        t2.account_type, \
                        t2.balance, \
                        t2.created_at AS user_message_time, \
                        t2.feedback AS resolution_status, \
                        t2.customer_type, \
                        t2.message AS user_input \
                        FROM recommender.executive_logs AS t1 \
                        INNER JOIN recommender.customer_logs AS t2 ON t1.executive_id = t2.executive_id \
                        WHERE abs(TIMESTAMPDIFF(SECOND, t1.created_at, t2.created_at)) <= 5")

        rows = cursor.fetchall()
        column_names = [i[0] for i in cursor.description]

        return rows, column_names

    except Error as e:
        print(e)

    finally:
        cursor.close()
        mydb.close()

In [35]:
logs, columns = query_with_fetchone()

In [36]:
d = {}
for i in tqdm(range(len(logs))):
    dummy = {}
    num = 0
    for el in logs[i]:
        # print(el, columns[num])
        if "executive_message_time" not in columns[num] and "executive_type" != columns[num]:
            if columns[num] == "user_message_time":
                dummy["weekday"] = el.weekday()
            else:
                dummy[columns[num]] = el
        num += 1

    d[i] = dummy

100%|██████████| 279445/279445 [00:01<00:00, 225234.32it/s]


In [37]:
def replace_accounts(x):
    if x == "Vista":
        return 1
    elif x == "Corriente":
        return 2
    elif x == "Ahorro":
        return 3
    else:
        return 4

def replace_type(x):
    if x == "Pyme":
        return 1
    elif x == "Business":
        return 2
    elif x == "Person":
        return 3
    elif x == "Others":
        return 4

In [8]:
dataset = pd.DataFrame.from_dict(d, "index")
dataset.resolution_status = dataset.resolution_status.apply(lambda x: 1 if "OK" == x else 0)
dataset.account_type = dataset.account_type.apply(replace_accounts)
dataset.customer_type = dataset.customer_type.apply(replace_type)

In [9]:
dataset.head()

,executed_action,account_type,balance,weekday,resolution_status,customer_type,user_input
0,Lleva a cabo consulta interna 14-23 por 12 - O...,2,623,0,1,1,Me puedes ayudar con una regularización en Yuanes
1,Lleva a cabo consulta interna 17-23 por 14,1,7,6,1,4,Quiero hacer una cajero automático en Rupías
2,Ingresa reclamo por 11,1,16790,0,0,2,Ayúdame con cuenta
3,Aplica operación por 4,1,16790,0,0,2,Ayúdame con cuenta
4,Ingresa reclamo por 11,1,16790,0,0,2,Ayúdame con cuenta


In [10]:
dataset.describe()

,account_type,balance,weekday,resolution_status,customer_type
count,279445.000000,279445.000000,279445.000000,279445.000000,279445.000000
mean,1.999975,8916.871091,2.770731,0.043819,2.119150
std,0.809093,22161.207209,1.981334,0.204693,1.009121
min,1.000000,1.000000,0.000000,0.000000,1.000000
25%,1.000000,7.000000,1.000000,0.000000,1.000000
50%,2.000000,92.000000,3.000000,0.000000,2.000000
75%,3.000000,909.000000,4.000000,0.000000,3.000000
max,3.000000,107831.000000,6.000000,1.000000,4.000000


In [11]:
dataset = dataset[dataset["resolution_status"] == 1]

In [12]:
hparams = {
    "batch_size": 128,
    "cnn_filter_sizes": [128, 128, 128],
    "cnn_kernel_sizes": [5, 5, 5],
    "cnn_pooling_sizes": [5, 5, 40],
    "constraint_learning_rate": 0.01,
    "embedding_dim": 100,
    "embedding_trainable": False,
    "learning_rate": 0.005,
    "max_num_words": 10000,
    "max_sequence_length": 250
}

In [13]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import text


In [14]:
tokenizer = text.Tokenizer(num_words=hparams["max_num_words"])
tokenizer.fit_on_texts(dataset["executed_action"] + dataset["user_input"].apply(lambda x: " " + x))

def prep_text(texts, tokenizer, max_sequence_length):
    # Turns text into into padded sequences.
    text_sequences = tokenizer.texts_to_sequences(texts)
    return sequence.pad_sequences(text_sequences, maxlen=max_sequence_length)

text_input = prep_text(dataset["user_input"], tokenizer, hparams["max_sequence_length"])
text_action = prep_text(dataset["executed_action"], tokenizer, hparams["max_sequence_length"])

In [15]:
train_dict = {"user_input": [], "metadata": [], "action_output": []}
test_dict = {"user_input": [], "metadata": [], "action_output": []}
counter = 0
for i, row in tqdm(dataset.iterrows()):
    input_data = list(text_input[counter])
    output_data = list(text_action[counter])
    metadata = []
    for n in range(246):
        metadata.append(0)
    metadata.append(row["balance"])
    metadata.append(row["weekday"])
    metadata.append(row["account_type"])
    metadata.append(row["customer_type"])

    if counter < len(dataset)*0.9:
        train_dict["user_input"].append(input_data)
        train_dict["metadata"].append(metadata)
        train_dict["action_output"].append(output_data)
    else:
        test_dict["user_input"].append(input_data)
        test_dict["metadata"].append(metadata)
        test_dict["action_output"].append(output_data)

    counter += 1

12245it [00:01, 8701.44it/s]


In [16]:
print(len(train_dict["user_input"]), len(test_dict["user_input"]))

11021 1224


In [31]:
def create_train_tfdata(train_feat_dict, train_target_tensor,
                        batch_size, buffer_size=None):
    """
    Create train tf dataset for model train input
    :param train_feat_dict: dict, containing the features tensors for train data
    :param train_target_tensor: np.array(), the training TARGET tensor
    :param batch_size: (int) size of the batch to work with
    :param buffer_size: (int) Optional. Default is None. Size of the buffer
    :return: (tuple) 1st element is the training dataset,
                     2nd is the number of steps per epoch (based on batch size)
    """
    if buffer_size is None:
        buffer_size = batch_size*50

    train_steps_per_epoch = len(train_target_tensor) // batch_size

    train_dataset = tf.data.Dataset.from_tensor_slices((train_feat_dict,
                                                        train_target_tensor)).cache()
    train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size)
    train_dataset = train_dataset.repeat().prefetch(tf.data.experimental.AUTOTUNE)
    
    return train_dataset, train_steps_per_epoch


train_feat_dict = {'user_input': train_dict['user_input'],
                    'metadata': train_dict['metadata']}
train_target_tensor = train_dict['action_output']

test_feat_dict = {'user_input': test_dict['user_input'],
                    'metadata': test_dict['metadata']}
                    
test_target_tensor = test_dict['action_output']

train_dataset, train_steps_per_epoch = create_train_tfdata(train_feat_dict,
                                                            train_target_tensor,
                                                            batch_size=128)

test_dataset, test_steps_per_epoch = create_train_tfdata(test_feat_dict,
                                                            test_target_tensor,
                                                            batch_size=128)

In [32]:
buffer_size = None
if buffer_size is None:
        buffer_size = 128*50

test_dataset = tf.data.Dataset.from_tensor_slices((test_feat_dict)).cache()
test_dataset = test_dataset.shuffle(buffer_size).batch(128)
test_dataset = test_dataset.repeat().prefetch(tf.data.experimental.AUTOTUNE)


In [19]:
test_target_tensor = tf.data.Dataset.from_tensor_slices((test_target_tensor)).cache()
# test_target_tensor = test_target_tensor.shuffle(buffer_size).batch(128)
# test_target_tensor = test_target_tensor.repeat().prefetch(tf.data.experimental.AUTOTUNE)

In [24]:
def build_model():
    """
    Build a model given the hyper-parameters with item and nb_days input features
    :param hp: (kt.HyperParameters) hyper-parameters to use when building this model
    :return: built and compiled tensorflow model 
    """
    item_vocab_size = 250
    inputs = {}
    inputs['user_input'] = tf.keras.Input(batch_input_shape=[None, 250],
                                       name='user_input', dtype=tf.int32)
    # create encoding padding mask
    encoding_padding_mask = tf.math.logical_not(tf.math.equal(inputs['user_input'], 0))

    # nb_days bucketized
    inputs['metadata'] = tf.keras.Input(batch_input_shape=[None, 250],
                                       name='metadata', dtype=tf.int32)

    # Pass categorical input through embedding layer
    # with size equals to tokenizer vocabulary size
    # Remember that vocab_size is len of item tokenizer + 1
    # (for the padding '0' value)
    
    embedding_item = tf.keras.layers.Embedding(input_dim= 2*item_vocab_size**2 + 1,
                                               output_dim=500,
                                               name='embedding_item'
                                              )(inputs['user_input'])
    # nbins=100, +1 for zero padding
    embedding_nb_days = tf.keras.layers.Embedding(input_dim=2*item_vocab_size**2 + 1,
                                                  output_dim=500,
                                                  name='embedding_nb_days'
                                                 )(inputs['metadata'])
    
    embedding_item = tf.keras.layers.GRU(500, return_sequences=True)(embedding_item)
    embedding_nb_days = tf.keras.layers.GRU(500, return_sequences=True)(embedding_nb_days)

    # embedding_item = tf.keras.layers.GRU(250, return_sequences=True)(embedding_item)
    # embedding_nb_days = tf.keras.layers.GRU(250, return_sequences=True)(embedding_nb_days)

    #  Concatenate embedding layers
    concat_embedding_input = tf.keras.layers.Concatenate(
     name='concat_embedding_input')([embedding_item, embedding_nb_days])

    concat_embedding_input = tf.keras.layers.BatchNormalization(
     name='batchnorm_inputs')(concat_embedding_input)
    
    # LSTM layer
    rnn =tf.keras.layers.Bidirectional( tf.keras.layers.LSTM(units=100,
                                   return_sequences=True,
                                   stateful=False,
                                   recurrent_initializer='glorot_normal',
                                   name='LSTM_cat_encoder'
                                   ))(concat_embedding_input)
    
    rnn =tf.keras.layers.Bidirectional( tf.keras.layers.LSTM(units=100,
                                   return_sequences=True,
                                   stateful=False,
                                   recurrent_initializer='glorot_normal',
                                   name='LSTM_cat_encoder_2'
                                   ))(rnn)

    rnn =tf.keras.layers.Bidirectional( tf.keras.layers.LSTM(units=100,
                                   return_sequences=True,
                                   stateful=False,
                                   recurrent_initializer='glorot_normal',
                                   name='LSTM_cat_encoder_3'
                                   ))(rnn)

    rnn = tf.keras.layers.BatchNormalization(name='batchnorm_lstm')(rnn)

    # Self attention so key=value in inputs
    att = tf.keras.layers.Attention(use_scale=False, causal=True,
                                    name='attention')(inputs=[rnn, rnn],
                                                      mask=[encoding_padding_mask,
                                                            encoding_padding_mask])

    # Last layer is a fully connected one
    # att = tf.keras.layers.Flatten()(att)

    output = tf.keras.layers.Dense(item_vocab_size, name='output')(att)
    
    # output = tf.keras.layers.Reshape(())

    model = tf.keras.Model(inputs, output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=loss_function,
        # loss = tf.keras.losses.BinaryCrossentropy(from_logits=False, name='binary_crossentropy'),
        metrics=['sparse_categorical_accuracy'])
    
    return model

In [25]:
def loss_function(real, pred):
    """
    We redefine our own loss function in order to get rid of the '0' value
    which is the one used for padding. This to avoid that the model optimize itself
    by predicting this value because it is the padding one.
    
    :param real: the truth
    :param pred: predictions
    :return: a masked loss where '0' in real (due to padding)
                are not taken into account for the evaluation
    """

    # to check that pred is numric and not nan
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_object_ = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                 reduction='none')
    loss_ = loss_object_(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [26]:
def fit_model(model, train_dataset, steps_per_epoch, epochs):
    """
    Fit the Keras model on the training dataset for a number of given epochs
    :param model: tf model to be trained
    :param train_dataset: (tf.data.Dataset object) the training dataset
                          used to fit the model
    :param steps_per_epoch: (int) Total number of steps (batches of samples) before 
                            declaring one epoch finished and starting the next epoch.
    :param epochs: (int) the number of epochs for the fitting phase
    :return: tuple (mirrored_model, history) with trained model and model history
    """
    
    # mirrored_strategy allows to use multi GPUs when available
    mirrored_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(
        tf.distribute.experimental.CollectiveCommunication.AUTO)
    
    with mirrored_strategy.scope():
        mirrored_model = model
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
    history = mirrored_model.fit(train_dataset,
                                 steps_per_epoch=steps_per_epoch,
                                 epochs=epochs, verbose=2, callbacks=[callback])

    return mirrored_model, history

In [27]:
mirror_model, history = fit_model(build_model(), train_dataset, 100, 100)

INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0',), communication = CommunicationImplementation.AUTO
Epoch 1/100
100/100 - 223s - loss: 0.0022 - sparse_categorical_accuracy: 0.2915
Epoch 2/100
100/100 - 189s - loss: 0.0017 - sparse_categorical_accuracy: 0.4585
Epoch 3/100
100/100 - 184s - loss: 0.0015 - sparse_categorical_accuracy: 0.5153
Epoch 4/100
100/100 - 186s - loss: 0.0014 - sparse_categorical_accuracy: 0.5578
Epoch 5/100
100/100 - 185s - loss: 0.0014 - sparse_categorical_accuracy: 0.5792
Epoch 6/100
100/100 - 189s - loss: 0.0013 - sparse_categorical_accuracy: 0.5899
Epoch 7/100
100/100 - 191s - loss: 0.0013 - sparse_categorical_accuracy: 0.5969
Epoch 8/100
100/100 - 190s - loss: 0.0013 - sparse_categorical_accuracy: 0.5965
Epoch 9/100
100/100 - 182s - loss: 0.0012 - sparse_categorical_accuracy: 0.6044
Epoch 10/100
100/100 - 194s - loss: 0.0012 - sparse_categoric

In [28]:
mirror_model.save("RNN_attention_deep_LSTM_v1.hdf5")

In [54]:
mirror_model = tf.keras.models.load_model("RNN_attention_GRU_v1.hdf5", custom_objects={"loss_function": loss_function})

In [29]:
from sklearn.metrics import accuracy_score

In [33]:
test_tensor = np.array(test_target_tensor)
counter = 0
acc = 0
denom = 0
for muestra in test_dataset:
    test = test_tensor[counter*(128 + 1) :(counter + 1)*128]
    pred = mirror_model.predict(muestra)
    predicted = np.argmax(pred, axis=1)
    # print(muestra)
    # print(np.argmax(pred, axis=1))
    # print(pred.shape)
    # print(np.argmax(pred, axis=1).shape)
    # print(test.shape)
    for row in range(test.shape[0]):
        # print(accuracy_score(predicted[row], test[row]))
        pp = predicted[row]
        tt = test[row]
        acc += accuracy_score(pp, tt)
        denom += 1
    counter += 1
    break
    
print(100*acc/denom)

49.22812500000002
